$$ \text{Explainability} $$
-- 
$$ \text{--} $$
$$ \text{Trend Following Strategy Implementation} $$
--

This notebook will explore explainability through the use of a Trend-Following strategy by using a Simple Moving Average(SMA) Cross Over Strategy with Meta-Labeling.

Marcos Lopez de Prado's approach to explainability is woven throughout his overall application of ML in finance. A few examples include the ability to understand data through proper labeling(dollar bars, tick bars), the ability to utilize inferential analysis by maximizing the amount of information retained in traditional time-series models like ARIMAs by using items such as fractional differentiation(), or the ability to come up with strategies separate from models so we're not relying on ML models to generate betting opportunities - instead only the ability to determine whether to pass on a bet or not. Further understanding/conviction of how he applies explainability can be built up from digging into his examples. 
 
One topic that I wanted to cover as a feature of explainability from Prado's book is 'Meta-Labeling', which is essentially a black box model on top of a white box. We will jump into Meta-Labeling below.


The majority of the functions were pulled from 'Advances in Financial Machine Learning' written by Marcos Lopez de Prado.<br>
Keep in mind that when working with time series data, the general tendency used here is to measure items in fixed chunks of time where we can apply statistical assumptions and use better models/more accurate inferences.<br>

Explainations of preceeding functions:
___
TRIPLE BARRIER METHOD & META-LABELING: 

The triple barrier method is used to label a fixed time horizon series. Labeling is an important aspect in explainability. It allows us to ensure that we are maximizing the understanding of our data before we place it in a model. Triple-Barrier-Method labels observations based on the first barrier touched,  then gives the target that was used to generate the horizontal barrier. Two horizontal barriers, profit taking(above) and stop-loss(below) limits; one vertical barrier(right side), time lapse in days or number of bars passed since first position. This will return one of three scenarios, which every barrier is touched first:
- Upper limit: 1
- Lower limit: -1
- Vertical limit: 0

Meta-Labeling separates the sides(Triple-Barrier) from the size. When our triple-barrier method's side is not None then we can use Meta-Labeling. Meta-Labeling discriminates between our profits and stop-loss orders. Now instead of getting a {1,0,-1} we get {1,0}. The ML algorithm will be trained to bet or pass. Then we can this primary model's result(signs) and use a secondary model to find the probabilities to decide on the size of the bet. Since the primary model is now binary we can filter out false positives and false negatives, and use a ROC curve to view the cost of increasing the true positive rate of a bet. First, we build a model with high recall. Then we correct the low precision by using met-labeling to the positives predicted by the model. In this we make our model more granular and can more readily ascertain why our model is making certain decisions. 


EVENT-BASED SAMPLINGS<br>
\- getTEvents: Detect shift in mean value of a measured quantity away from a target value.<br>
COMPUTING DYNAMIC THRESHOLDS<br>
\- getDailyVol:  set profit taking and stoploss limits that are a function of the risks involved in a bet<br>
TRIPLE-BARRIER METHOD<br>
\- applyPtSlOnT1: label an observation according to the first barrier touched out of three barriers<br>
LEARNING SIDE AND SIZE<br>
\- getEvents: learn sides(to go long or short on a bet), learn size(how much to risk)<br>
ADDING A VERTICAL BARRIER<br>
\- addVerticalBarrier: For each index in tEvents, get timestamp of next price bar at or immediately after a number of days<br>
LABELING FOR SIDE AND SIZE<br>
\- getBins: Label observations for return at time of first barrier touched, and {-1,0,1}<br>
DROPPING UNDER-POPULATED LABELS<br>
\- dropLabels: drop observations less than significant % of total classes, ie. 5%
___
LABELS:

STANDARD BARS<br>
\- general_bars: generates base function for standard bars to create dollar bars and tick bars.<br>
STANDARD BARS DF<br>
\- general_bar_df: turns general_bars to dataframe<br>
DOLLAR BARS<br>
\- dollar_bar_df: formed by sampling an observation every time a pre-defined market value is exchanged.<br>
TICK BARS<br>
\- tick_bars: extracted each time a pre-defined number of transactions takes place.

___
Can mainly ignore this.. didn't have time to fully go through the functions to simplify them. 

PROCESSING SPEEDS:

ATOMS AND MOLECULES - Atoms are indivisible tasks i.e. single threaded, molecules are grouped i.e. parallel processing. 

LINEAR PARTITIONS<br>
\- linParts: Create list of atoms in subsets of equal size, proportional to number of processors. <br>
TWO-NESTED LOOPS PARTITIONS<br>
\- nestedParts: parallelize nested functions<br>
MULTIPROCESSING ENGINE<br>
\- mpPandasObj: parallelization wrapper<br>
SINGLE-THREAD EXECUTION, FOR DEBUGGING<br>
\- processJobs_: runs jobs sequentially<br>
CENTAGE OF JOBS COMPLETED<br>
\- reportProgress: shows progress for number of asynchronous jobs.<br>
ASYNCHRONOUS CALL<br>
\- processJobs: run jobs in parallel, and process back into single output if needed. <br>
UNWRAPPING THE CALLBACK<br>
\- expandCall: executes each job(each thread). Turns dictionary into task. <br>

___
Import libraries and create functions:
___

In [1]:
import re, os, sys, time, types #!pip install pyarrow
import warnings
import numpy as np
import pandas as pd # Don't use pandas 1.0.0
import datetime as dt
from tqdm import tqdm
import multiprocessing as mp
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, classification_report

warnings.filterwarnings("ignore")
%matplotlib inline

In [2]:
## Symmetric CUSUM Filter
def getTEvents(gRaw, h):
    '''
    Symmetric CUSUM Filter
    Sample a bar t iff S_t >= h at which point S_t is reset
    Multiple events are not triggered by gRaw hovering around a threshold level
    It will require a full run of length h for gRaw to trigger an event
    gRaw: the raw time series we wish to filter (gRaw)
    h: threshold
    Return: pd.DatatimeIndex.append(tEvents)
    '''
    tEvents, sPos, sNeg = [], 0, 0
    diff = np.log(gRaw).diff().dropna()
    for i in tqdm(diff.index[1:]):
        try:
            pos, neg = float(sPos+diff.loc[i]), float(sNeg+diff.loc[i])
        except Exception as e:
            print(e)
            print(sPos+diff.loc[i], type(sPos+diff.loc[i]))
            print(sNeg+diff.loc[i], type(sNeg+diff.loc[i]))
            break
        sPos, sNeg=max(0., pos), min(0., neg)
        if sNeg<-h:
            sNeg=0;tEvents.append(i)
        elif sPos>h:
            sPos=0;tEvents.append(i)
    return pd.DatetimeIndex(tEvents)  

In [3]:
## Daily Volatility Estimator
def getDailyVol(close, span0 = 100):
    '''
    Compute the daily volatility at intraday estimation
    applying a span of span0 to an exponentially weighted moving standard deviation
    Set profit taking and stop loss limits that are function of the risks involved in a bet
    '''
    df0=close.index.searchsorted(close.index-pd.Timedelta(days=1))
    df0=df0[df0>0]   
    df0=(pd.Series(close.index[df0-1], index=close.index[close.shape[0]-df0.shape[0]:]))   
    try:
        df0=close.loc[df0.index]/close.loc[df0.values].values-1 # daily rets
    except Exception as e:
        print(f'error: {e}\nplease confirm no duplicate indices')
    df0=df0.ewm(span=span0).std().rename('dailyVol')
    return df0

In [4]:
def applyPtSlOnT1(close,events,ptSl,molecule):
    # apply stop loss/profit taking, if it takes place before t1 (end of event)
    events_=events.loc[molecule]
    out=events_[['t1']].copy(deep=True)
    if ptSl[0]>0:
        pt=ptSl[0]*events_['trgt']
    else: 
        pt=pd.Series(index=events.index) # NaNs
    if ptSl[1]>0:
        sl=-ptSl[1]*events_['trgt']
    else:
        sl=pd.Series(index=events.index) # NaNs
    
    for loc,t1 in events_['t1'].fillna(close.index[-1]).iteritems():
        df0=close[loc:t1] # path prices
        df0=(df0/close[loc]-1)*events_.at[loc,'side'] # path returns
        out.loc[loc,'sl']=df0[df0<sl[loc]].index.min() # earliest stop loss
        out.loc[loc,'pt']=df0[df0>pt[loc]].index.min() # earliest profit taking
    return out

In [5]:
def getEvents(close, tEvents, ptSl, trgt, minRet, numThreads, t1=False, side=None):
    #1) get target
    trgt=trgt.loc[tEvents]
    trgt=trgt[trgt>minRet] # minRet
    #2) get t1 (max holding period)
    if t1 is False:t1=pd.Series(pd.NaT, index=tEvents)
    #3) form events object, apply stop loss on t1
    if side is None:side_,ptSl_=pd.Series(1.,index=trgt.index), [ptSl[0],ptSl[0]]
    else: side_,ptSl_=side.loc[trgt.index],ptSl[:2]
    events=(pd.concat({'t1':t1,'trgt':trgt,'side':side_}, axis=1)
            .dropna(subset=['trgt']))
    df0=mpPandasObj(func=applyPtSlOnT1,pdObj=('molecule',events.index),
                    numThreads=numThreads,close=close,events=events,
                    ptSl=ptSl_)
    events['t1']=df0.dropna(how='all').min(axis=1) # pd.min ignores nan
    if side is None:events=events.drop('side',axis=1)
    return events

In [6]:
def addVerticalBarrier(tEvents, close, numDays=1):
    t1=close.index.searchsorted(tEvents+pd.Timedelta(days=numDays))
    t1=t1[t1<close.shape[0]]
    t1=(pd.Series(close.index[t1],index=tEvents[:t1.shape[0]]))
    return t1

In [7]:
def getBins(events, close):
    '''
    Compute event's outcome (including side information, if provided). events is a DataFrame where: 
    —events.index is event's starttime 
    —events[’t1’] is event's endtime 
    —events[’trgt’] is event's target 
    —events[’side’] (optional) implies the algo's position side 
    Case 1: (’side’ not in events): bin in (-1,1) <—label by price action 
    Case 2: (’side’ in events): bin in (0,1) <—label by pnl (meta-labeling)
    '''
    #1) prices aligned with events
    events_=events.dropna(subset=['t1'])
    px=events_.index.union(events_['t1'].values).drop_duplicates()
    px=close.reindex(px,method='bfill')
    #2) create out object
    out=pd.DataFrame(index=events_.index)
    out['ret']=px.loc[events_['t1'].values].values/px.loc[events_.index]-1
    if 'side' in events_:out['ret']*=events_['side'] # meta-labeling
    out['bin']=np.sign(out['ret'])
    if 'side' in events_:out.loc[out['ret']<=0,'bin']=0 # meta-labeling
    return out

In [8]:
def dropLabels(events, minPct=.05):
    # apply weights, drop labels with insufficient examples
    while True:
        df0=events['bin'].value_counts(normalize=True)
        if df0.min()>minPct or df0.shape[0]<3:break
        print('dropped label: ', df0.argmin(),df0.min())
        events=events[events['bin']!=df0.argmin()]
    return events

In [9]:
def linParts(numAtoms,numThreads):
    # partition of atoms with a single loop
    parts=np.linspace(0,numAtoms,min(numThreads,numAtoms)+1)
    parts=np.ceil(parts).astype(int)
    return parts

def nestedParts(numAtoms,numThreads,upperTriang=False):
    # partition of atoms with an inner loop
    parts,numThreads_=[0],min(numThreads,numAtoms)
    for num in range(numThreads_):
        part=1+4*(parts[-1]**2+parts[-1]+numAtoms*(numAtoms+1.)/numThreads_)
        part=(-1+part**.5)/2.
        parts.append(part)
    parts=np.round(parts).astype(int)
    if upperTriang: # the first rows are heaviest
        parts=np.cumsum(np.diff(parts)[::-1])
        parts=np.append(np.array([0]),parts)
    return parts

In [10]:
def mpPandasObj(func,pdObj,numThreads=24,mpBatches=1,linMols=True,**kargs):
    '''
    Parallelize jobs, return a dataframe or series
    + func: function to be parallelized. Returns a DataFrame
    + pdObj[0]: Name of argument used to pass the molecule
    + pdObj[1]: List of atoms that will be grouped into molecules
    + kwds: any other argument needed by func
    Example: df1=mpPandasObj(func,('molecule',df0.index),24,**kwds)
    '''
    if linMols:parts=linParts(len(pdObj[1]),numThreads*mpBatches)
    else:parts=nestedParts(len(pdObj[1]),numThreads*mpBatches)
    
    jobs=[]
    for i in range(1,len(parts)):
        job={pdObj[0]:pdObj[1][parts[i-1]:parts[i]],'func':func}
        job.update(kargs)
        jobs.append(job)
    out=processJobs(jobs,numThreads=numThreads)
    if isinstance(out[0],pd.DataFrame):df0=pd.DataFrame()
    elif isinstance(out[0],pd.Series):df0=pd.Series()
    else:return out
    for i in out:df0=df0.append(i)
    df0=df0.sort_index()
    return df0

def processJobs_(jobs):
    # Run jobs sequentially, for debugging
    out=[]
    for job in jobs:
        out_=expandCall(job)
        out.append(out_)
    return out

In [11]:
def reportProgress(jobNum,numJobs,time0,task):
    # Report progress as asynch jobs are completed
    msg=[float(jobNum)/numJobs, (time.time()-time0)/60.]
    msg.append(msg[1]*(1/msg[0]-1))
    timeStamp=str(dt.datetime.fromtimestamp(time.time()))
    msg=timeStamp+' '+str(round(msg[0]*100,2))+'% '+task+' done after '+ \
        str(round(msg[1],2))+' minutes. Remaining '+str(round(msg[2],2))+' minutes.'
    if jobNum<numJobs:sys.stderr.write(msg+'\r')
    else:sys.stderr.write(msg+'\n')
    return

def processJobs(jobs,task=None,numThreads=24):
    # Run in parallel.
    # jobs must contain a 'func' callback, for expandCall
    if task is None:task=jobs[0]['func'].__name__
    pool=mp.Pool(processes=numThreads)
    outputs,out,time0=pool.imap_unordered(expandCall,jobs),[],time.time()
    # Process asyn output, report progress
    for i,out_ in enumerate(outputs,1):
        out.append(out_)
        reportProgress(i,len(jobs),time0,task)
    pool.close();pool.join() # this is needed to prevent memory leaks
    return out

def expandCall(kargs):
    # Expand the arguments of a callback function, kargs['func']
    func=kargs['func']
    del kargs['func']
    out=func(**kargs)
    return out

In [12]:
def general_bars(df,column,m,tick = False):
    t = df[column]
    ts = 0
    idx = []
    if tick: # if tick bar
        for i,x in enumerate(t):
            ts += 1 # each data plus 1
            if ts > m:
                idx.append(i)
                ts = 0
    else: # if not tick bar
        for i,x in enumerate(t):
            ts += x # each data plus volume/dollar volume
            if ts > m:
                idx.append(i)
                ts = 0
    return idx

def general_bar_df(df,column,m, tick = False):
    idx = general_bars(df, column, m, tick)
    df = df.iloc[idx].drop_duplicates()
    df['dates'] = df['dates'] + pd.to_timedelta(df.groupby('dates').cumcount(), unit='ms')
    return df

def dollar_bar_df(df,dollar_column,m):
    return general_bar_df(df,dollar_column,m)

def tick_bars(df,price_column,m):
    return general_bars(df,price_column,m, tick = True)

___
Import data and run code:
___

In [13]:
df = pd.read_parquet('IVE_dollarValue_resampled_1s.parquet')
df['dates'] = df.index

Dollar bars allow us to label our data to retain more information, there are many other options:

In [14]:
# Dollar Bars
dbars = dollar_bar_df(df,'dv',1_000_000).drop_duplicates().dropna() # Measure our data
close = dbars.price.copy()
dailyVol = getDailyVol(close) # Measure our risk
tEvents = getTEvents(close, h = dailyVol.mean()) #Given a time series and a threshold(profit/stop order), allows for event based sampling. Think bollinger banads but not activated when overs around event threshold, take into account a time frame. 
t1 = addVerticalBarrier(tEvents,close) # Vertical barrier - max length of time bet will be run.

100%|██████████| 30859/30859 [00:01<00:00, 20255.73it/s]


In [15]:
# Triple-barrier:

# Target Series
ptSl = [1,1] # factor that multiplies trgt to set the width of the upper barrier. If 0, there will not be an upper barrier
target = dailyVol # Risk tolerance
minRet = 0.01 # minimum acceptable return required for running triple barrier method
events = getEvents(close,tEvents, ptSl, target, minRet,1, t1=t1) # finds the time of the first barrier touch
labels = getBins(events, close) # getBins - generate labels
clean_labels = dropLabels(labels) # drop under-populated labels

2020-02-05 14:05:18.611525 100.0% applyPtSlOnT1 done after 0.03 minutes. Remaining 0.0 minutes.


In [16]:
# Moving Average Crossover Strategy. - Model gives side but not size of bet. Aka our primary model
fast_window = 3
slow_window = 7
close_df = (pd.DataFrame().assign(price = close).assign(fast=close.ewm(fast_window).mean()).assign(slow =close.ewm(slow_window).mean()))

def get_up_cross(df):
    crit1 = df.fast.shift(1) < df.slow.shift(1)
    crit2 = df.fast > df.slow
    return df.fast[(crit1)&(crit2)]

def get_down_cross(df):
    crit1 = df.fast.shift(1) > df.slow.shift(1)
    crit2 = df.fast < df.slow
    return df.slow[(crit1) & (crit2)]

up = get_up_cross(close_df)
down = get_down_cross(close_df)

In [17]:
# Meta-labels: our secondary model
# Trading signal
side_up = pd.Series(1,index = up.index)
side_down = pd.Series(-1, index = down.index)
side = pd.concat([side_up, side_down]).sort_index()
minRet = .01 # minimum acceptable return required for running triple barrier method
ptSl = [1,2] # factor that multiplies trgt to set the width of the upper barrier. If 0, there will not be an upper barrier
ma_events = getEvents(close, tEvents, ptSl, target, minRet, 1, t1 = t1, side = side) # finds the time of the first barrier touch for simple moving average trend following strategy

2020-02-05 14:05:21.594629 100.0% applyPtSlOnT1 done after 0.04 minutes. Remaining 0.0 minutes.


In [18]:
ma_side= ma_events.dropna().side # Drop nulls, get sides
ma_bins = getBins(ma_events,close).dropna()  # getBins - generate labels
Xx = pd.merge_asof(ma_bins, side.to_frame().rename(columns={0:'side'}), left_index=True, right_index=True, direction='forward') # Merge side with labels


In [19]:
Xx.ret.sum()

0.14388169456817856

Train Random Forest to choose to trade or not {0,1} since undelying model, Cross SMA, has decided the side, {-1,1}.

In [20]:
X = ma_side.values.reshape(-1,1) # sides
y = ma_bins.bin.values # Labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, shuffle=False)
n_estimator = 10000
rf = RandomForestClassifier(max_depth=2, n_estimators=n_estimator, criterion='entropy', random_state=42)
rf.fit(X_train, y_train);

# The random forest model by itself
y_pred_rf = rf.predict_proba(X_test)[:,1]
y_pred = rf.predict(X_test)
fpr_rf, tpr_rf, _ = roc_curve(y_test, y_pred_rf)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        24
         1.0       0.53      1.00      0.69        27

    accuracy                           0.53        51
   macro avg       0.26      0.50      0.35        51
weighted avg       0.28      0.53      0.37        51



Although our results aren't necessarily ideal, we view the process of breaking down a typical machine learning model, and provide a preview of the current state of affairs for explainability in applications of machine learning in finance. 